In [1]:
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Ridge, HuberRegressor
from sklearn.feature_selection import RFE




In [4]:
train_stacked = pd.read_csv('oofs/kain-train-features-v0.1.2.csv')
test_stacked = pd.read_csv('oofs/kain-test-features-v0.1.2.csv')

In [10]:
train = pd.read_csv('/media/limbo/Home-Credit/data/application_train.csv.zip')
y = train['TARGET']

n_train = train.shape[0]

In [11]:
train_features = train_stacked.values
test_features = test_stacked.values

In [12]:
model = Ridge(alpha=100, max_iter=1000, fit_intercept=False,
                            random_state=666, normalize=False)

In [13]:
selector = RFE(model, 105, step=2)
selector = selector.fit(train_features, y)

In [14]:
selected_features = [i for i, y in enumerate(selector.ranking_) if y == 1]

In [15]:
selected_features

[1,
 2,
 3,
 4,
 5,
 8,
 10,
 12,
 13,
 14,
 17,
 18,
 19,
 20,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 34,
 35,
 39,
 40,
 41,
 43,
 44,
 46,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 74,
 75,
 76,
 77,
 79,
 82,
 83,
 84,
 85,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 97,
 102,
 103,
 104,
 105,
 107,
 109,
 111,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 127,
 128,
 130,
 131,
 133,
 135,
 136,
 137,
 139,
 140,
 141,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153]

In [16]:
train_features.shape

(307511, 155)

In [17]:
aucs = []
test_set = []
validation_set = []
print("\nModeling Stage")

kf = KFold(n_splits=5, random_state=1002, shuffle=True)
kf.get_n_splits(train_features)

n_bagged = 6

for train_index, test_index in kf.split(train_features):
    print("TRAIN: ", train_index, "TEST: ", test_index)

    X = train_features
    y_ = y.values
    x_train, x_test = X[train_index][:, selected_features], X[test_index][:, selected_features]
    y_train, y_test = y_[train_index], y_[test_index]
    
    oof_baggs = np.zeros([n_bagged , x_test.shape[0]])
    preds_baggs = np.zeros([n_bagged , test_features.shape[0]])
    
    for _it in range(1, n_bagged):
        
        model = Ridge(alpha= min(4 **_it, 333), max_iter=1000, fit_intercept=False,
                                random_state=666, normalize=False)
        model.fit(x_train, y_train)
        
        print(_it,' ' ,'Fold AUC :', roc_auc_score(y_test, model.predict(x_test)))
        oof_baggs[_it, :] = model.predict(x_test)
        preds_baggs[_it, :] = model.predict(test_features[:, selected_features])
        
    val_preds = pd.DataFrame(oof_baggs).T
    test_preds = pd.DataFrame(preds_baggs).T
    
    print('Fold AUC :', roc_auc_score(y_test, val_preds.rank(axis=0, method='min').mul(val_preds.shape[1] * [1 / val_preds.shape[1]]).sum(1) / val_preds.shape[0]))
    aucs.append(roc_auc_score(y_test, val_preds.rank(axis=0, method='min').mul(val_preds.shape[1] * [1 / val_preds.shape[1]]).sum(1) / val_preds.shape[0]))
   
    test_set.append(test_preds.rank(axis=0, method='min').mul(test_preds.shape[1] * [1 / test_preds.shape[1]]).sum(1) / test_preds.shape[0])
    gc.collect()
        
print('AVERAGED AUC :', np.mean(aucs))


Modeling Stage
TRAIN:  [     0      1      2 ... 307505 307506 307508] TEST:  [     9     16     25 ... 307507 307509 307510]
1   Fold AUC : 0.8039375927019463
2   Fold AUC : 0.8040660362545016
3   Fold AUC : 0.8040894845494969
4   Fold AUC : 0.803591282734116
5   Fold AUC : 0.803423169043922
Fold AUC : 0.804043117205644
TRAIN:  [     0      2      3 ... 307508 307509 307510] TEST:  [     1      8     17 ... 307497 307503 307504]
1   Fold AUC : 0.8079275314782856
2   Fold AUC : 0.8081071864675232
3   Fold AUC : 0.8082365749394144
4   Fold AUC : 0.8080828545486871
5   Fold AUC : 0.8080074349642672
Fold AUC : 0.8082833677836232
TRAIN:  [     0      1      4 ... 307507 307509 307510] TEST:  [     2      3      6 ... 307494 307501 307508]
1   Fold AUC : 0.8064498681103255
2   Fold AUC : 0.8067536198567467
3   Fold AUC : 0.8070723908650327
4   Fold AUC : 0.8070051034318959
5   Fold AUC : 0.8069273032279425
Fold AUC : 0.8070849271037668
TRAIN:  [     0      1      2 ... 307508 307509 307510

In [224]:
test_preds = pd.DataFrame(test_set).T

In [123]:
pd.DataFrame({'folds': range(5), 'aucs': aucs}).to_csv('folds/ridge-folds.csv', index=False)

In [124]:

y_hat = test_preds.rank(axis=0, method='min').mul(test_preds.shape[1] * [1 / test_preds.shape[1]]).sum(1) / test_preds.shape[0] 
    


# In[82]:


y_hat.shape


# In[83]:


y_hat.head(10)


# In[84]:


sampl_sub = pd.read_csv('../data/sample_submission.csv')


# In[86]:


sampl_sub['TARGET'] = y_hat.clip(0, 1).values


# In[ ]:


sampl_sub.to_csv("ridge-stack-submission.csv", index=False)


# In[ ]:


sampl_sub.head()



,SK_ID_CURR,TARGET
0,100001,0.356208
1,100005,0.835775
2,100013,0.407283
3,100028,0.379887
4,100038,0.787982
